# Importações

In [1]:
import pandas as pd 
import numpy as np
import re
from utilsforecast.preprocessing import fill_gaps

# Importar dados do PostgreSQL

In [2]:
# Como a Api ainda não esta a funcionar, vou importar os dados de um csv
df = pd.read_csv('../all_bike_counts.csv', parse_dates=['detected'], dtype='int32')
df = df.set_index('detected')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22632 entries, 2021-08-01 00:00:00 to 2024-02-29 23:00:00
Data columns (total 62 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   direction_2   22632 non-null  int32
 1   count_2       22632 non-null  int32
 2   direction_3   22632 non-null  int32
 3   count_3       22632 non-null  int32
 4   direction_4   22632 non-null  int32
 5   count_4       22632 non-null  int32
 6   direction_5   22632 non-null  int32
 7   count_5       22632 non-null  int32
 8   direction_6   22632 non-null  int32
 9   count_6       22632 non-null  int32
 10  direction_7   22632 non-null  int32
 11  count_7       22632 non-null  int32
 12  direction_8   22632 non-null  int32
 13  count_8       22632 non-null  int32
 14  direction_9   22632 non-null  int32
 15  count_9       22632 non-null  int32
 16  direction_10  22632 non-null  int32
 17  count_10      22632 non-null  int32
 18  direction_11  22632 non-null  i

In [3]:
df_loc = pd.read_csv('../all_counter_locations.csv')
df_loc.info()
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   locationId         31 non-null     int64  
 1   tenantIdentifier   31 non-null     object 
 2   name               31 non-null     object 
 3   cp7                31 non-null     object 
 4   freguesia          31 non-null     object 
 5   directionPositive  0 non-null      float64
 6   directionNegative  0 non-null      float64
 7   latitude           31 non-null     float64
 8   longitude          31 non-null     float64
dtypes: float64(4), int64(1), object(4)
memory usage: 2.3+ KB



# Analises

In [4]:
df.index

DatetimeIndex(['2021-08-01 00:00:00', '2021-08-01 01:00:00',
               '2021-08-01 02:00:00', '2021-08-01 03:00:00',
               '2021-08-01 04:00:00', '2021-08-01 05:00:00',
               '2021-08-01 06:00:00', '2021-08-01 07:00:00',
               '2021-08-01 08:00:00', '2021-08-01 09:00:00',
               ...
               '2024-02-29 14:00:00', '2024-02-29 15:00:00',
               '2024-02-29 16:00:00', '2024-02-29 17:00:00',
               '2024-02-29 18:00:00', '2024-02-29 19:00:00',
               '2024-02-29 20:00:00', '2024-02-29 21:00:00',
               '2024-02-29 22:00:00', '2024-02-29 23:00:00'],
              dtype='datetime64[ns]', name='detected', length=22632, freq=None)

In [5]:
df.loc['2022'].tail(2)

,direction_2,count_2,direction_3,count_3,direction_4,count_4,direction_5,count_5,direction_6,count_6,...,direction_31,count_31,direction_32,count_32,direction_33,count_33,direction_34,count_34,direction_35,count_35
detected,,,,,,,,,,,,,,,,,,,,,
2022-12-31 22:00:00,0,7,2,2,2,2,2,5,9,15,...,1,1,1,15,26,127,52,87,12,55
2022-12-31 23:00:00,1,11,0,1,1,3,5,6,10,14,...,2,2,4,14,15,238,99,138,24,100


In [6]:
df.resample('D').first().dropna().index[[0, -1]]

DatetimeIndex(['2021-08-01', '2024-02-29'], dtype='datetime64[ns]', name='detected', freq=None)

In [7]:
df.resample('D').first().dropna().iloc[[0, -1]]

,direction_2,count_2,direction_3,count_3,direction_4,count_4,direction_5,count_5,direction_6,count_6,...,direction_31,count_31,direction_32,count_32,direction_33,count_33,direction_34,count_34,direction_35,count_35
detected,,,,,,,,,,,,,,,,,,,,,
2021-08-01,0,0,1,1,0,0,0,0,6,7,...,1,1,3,7,4,6,2,2,30,42
2024-02-29,6,7,0,1,4,6,1,7,3,12,...,1,3,2,5,8,11,2,13,5,17


In [8]:
# Seleciona o primeiro e o último dia com os contadores
df_mostrar = df.resample('D').first().dropna().iloc[[0, -1]]

# Identifica as colunas que representam os contadores e direções
contador_cols = [col for col in df.columns if col.startswith('direction_') or col.startswith('count_')]

# Identifica o primeiro e o último contador
primeiro_contador = sorted(set(int(col.split('_')[1]) for col in contador_cols))[0]
ultimo_contador = sorted(set(int(col.split('_')[1]) for col in contador_cols))[-1]

# Seleciona as colunas para o primeiro e o último contador
primeiro_contador_cols = [f'direction_{primeiro_contador}', f'count_{primeiro_contador}']
ultimo_contador_cols = [f'direction_{ultimo_contador}', f'count_{ultimo_contador}']

# Exibe as colunas desejadas
df_selecionado = df_mostrar[primeiro_contador_cols + ultimo_contador_cols]

# Exibe o resultado
df_selecionado


,direction_2,count_2,direction_35,count_35
detected,,,,
2021-08-01,0,0,30,42
2024-02-29,6,7,5,17


In [9]:
df_loc.head(2)

,locationId,tenantIdentifier,name,cp7,freguesia,directionPositive,directionNegative,latitude,longitude
0,2,lisboa.pt.mycycletraffic,#18 - Avenida do Colégio Militar,1500-069,Benfica,NaN,NaN,38.749623,-9.191153
1,3,lisboa.pt.mycycletraffic,#01 - Telheiras - Colégio Alemão,1600-616,Lumiar,NaN,NaN,38.759682,-9.164044


### Identificar se há dias com valores a zéro

In [10]:
df_daily = df.resample('D').sum()

In [ ]:
# import re

# def analisar_zeros(df_daily, contadores, df_loc):
#     resultados = []

#     for contador in contadores:
#         if contador in df_daily.columns:
#             series = df_daily[contador].copy()
#             zero_days = series[series == 0]

#             if not zero_days.empty:
#                 match = re.match(r'count_(\d+)', contador)
#                 if match:
#                     location_id = int(match.group(1))
#                     try:
#                         location_name = df_loc.query("locationId == @location_id").name.values[0]
#                     except IndexError:
#                         location_name = "Nome desconhecido"

#                     diffs = zero_days.index.to_series().diff().dt.days.fillna(1)
#                     grupo = (diffs != 1).cumsum()
#                     sequencias = zero_days.groupby(grupo)

#                     detalhes_sequencias = []
#                     for _, seq in sequencias:
#                         detalhes_sequencias.append({
#                             'inicio': seq.index[0].date(),
#                             'fim': seq.index[-1].date(),
#                             'dias': len(seq)
#                         })

#                     resultados.append({
#                         'contador': contador,
#                         'nome': location_name,
#                         'total_dias_zeros': len(zero_days),
#                         'n_sequencias': len(detalhes_sequencias),
#                         'sequencias': detalhes_sequencias
#                     })

#     return resultados




In [41]:
def analisar_zeros_simples(df_daily, df_loc):
    resultados = []

    for col in df_daily.columns:
        if col.startswith('count_'):
            zero_days = df_daily[df_daily[col] == 0].index

            # Obter o nome do local (se disponível)
            location_id = int(col.split('_')[1])
            location_name = df_loc[df_loc["locationId"] == location_id]["name"].values
            location_name = location_name[0] if len(location_name) > 0 else "Nome desconhecido"

            if len(zero_days) == 0:
                resultados.append({
                    'contador': col,
                    'nome': location_name,
                    'sequencias': []
                })
            else:
                zero_days_series = pd.Series(zero_days)
                diffs = zero_days_series.diff().dt.days.fillna(1)
                grupo = (diffs != 1).cumsum()

                sequencias = []
                for _, seq in zero_days_series.groupby(grupo):
                    sequencias.append({
                        'inicio': seq.iloc[0].date(),
                        'fim': seq.iloc[-1].date(),
                        'dias': len(seq)
                    })

                resultados.append({
                    'contador': col,
                    'nome': location_name,
                    'sequencias': sequencias
                })

    return resultados


In [43]:
resultados = analisar_zeros_simples(df_daily, df_loc)

for r in resultados:
    print(f"\nContador: {r['contador']} ({r['nome']})")
    if not r['sequencias']:
        print(" Sem dias com valor zero.")
    else:
        print(f"Número de sequências: {len(r['sequencias'])}")
        for seq in r['sequencias']:
            print(f" - De {seq['inicio']} a {seq['fim']} ({seq['dias']} dias)")



Contador: count_2 (#18 - Avenida do Colégio Militar)
Número de sequências: 4
 - De 2022-08-20 a 2022-08-28 (9 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_3 (#01 - Telheiras - Colégio Alemão)
Número de sequências: 5
 - De 2022-08-18 a 2022-08-28 (11 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2023-12-01 a 2023-12-04 (4 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_4 (#02 - Avenida do Brasil - Mata de Alvalade)
Número de sequências: 4
 - De 2022-08-18 a 2022-08-28 (11 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_5 (#03 - Avenida do Brasil - Campo Grande)
Número de sequências: 4
 - De 2022-08-20 a 2022-08-28 (9 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contad

In [ ]:
# contadores = [col for col in df_daily.columns if col.startswith('count_')]
# resultados_zeros = analisar_zeros(df_daily, contadores, df_loc)

# for r in resultados_zeros:
#     print(f"\nContador: {r['contador']} ({r['nome']})")
#     print(f"Total de dias com zeros: {r['total_dias_zeros']}")
#     print(f"Número de sequências: {r['n_sequencias']}")
#     for seq in r['sequencias']:
#         print(f" - De {seq['inicio']} a {seq['fim']} ({seq['dias']} dias)")



Contador: count_2 (#18 - Avenida do Colégio Militar)
Total de dias com zeros: 14
Número de sequências: 4
 - De 2022-08-20 a 2022-08-28 (9 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_3 (#01 - Telheiras - Colégio Alemão)
Total de dias com zeros: 20
Número de sequências: 5
 - De 2022-08-18 a 2022-08-28 (11 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2023-12-01 a 2023-12-04 (4 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_4 (#02 - Avenida do Brasil - Mata de Alvalade)
Total de dias com zeros: 16
Número de sequências: 4
 - De 2022-08-18 a 2022-08-28 (11 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_5 (#03 - Avenida do Brasil - Campo Grande)
Total de dias com zeros: 14
Número de sequências: 4
 - De 2022-08-20 a 2022-08-28 (9 dias)
 - De 2023-0

### Fill gaps 

In [25]:
def fill_zeros(df_daily, max_weeks_back=5):
    df_filled = df_daily.copy()
    for col in df_daily.columns: 
        if col.startswith('count_'): 
            zero_dates = df_daily[df_daily[col]==0].index
            for datetime in zero_dates:
                 for weeks_back in range(1, max_weeks_back + 1):
                    prev_week = datetime - pd.Timedelta(weeks=weeks_back)
                    if prev_week in df_daily.index:
                        prev_value = df_daily.loc[prev_week, col]
                        if prev_value != 0:
                            df_filled.loc[datetime, col] = prev_value
                            break
    return df_filled

In [26]:
df_daily_filled = fill_zeros(df_daily)

In [33]:
print(df_daily_filled.loc['2022-08-21'])

direction_2        0
count_2          679
direction_3        0
count_3          159
direction_4        0
                ... 
count_33        1063
direction_34       0
count_34        1638
direction_35       0
count_35        1036
Name: 2022-08-21 00:00:00, Length: 62, dtype: int32


In [45]:
resultados = analisar_zeros_simples(df_daily_filled, df_loc)

for r in resultados:
    print(f"\nContador: {r['contador']} ({r['nome']})")
    if not r['sequencias']:
        print("Sem dias com valor zero.")
    else:
        print(f"Número de sequências: {len(r['sequencias'])}")
        for seq in r['sequencias']:
            print(f" - De {seq['inicio']} a {seq['fim']} ({seq['dias']} dias)")




Contador: count_2 (#18 - Avenida do Colégio Militar)
Sem dias com valor zero.

Contador: count_3 (#01 - Telheiras - Colégio Alemão)
Sem dias com valor zero.

Contador: count_4 (#02 - Avenida do Brasil - Mata de Alvalade)
Sem dias com valor zero.

Contador: count_5 (#03 - Avenida do Brasil - Campo Grande)
Sem dias com valor zero.

Contador: count_6 (#04 - Entrecampos - Campo Grande/Av. Forças Armadas)
Sem dias com valor zero.

Contador: count_7 (#05 - Av. Duque D'Ávila/Pç. Duque Saldanha)
Sem dias com valor zero.

Contador: count_8 (#11 - Avenida Lusíada (Entrada - Benfica))
Sem dias com valor zero.

Contador: count_9 (#31 - Avenida Brasília - Champalimaud)
Sem dias com valor zero.

Contador: count_10 (#19 - Avenida General Correia Barreto (Radial Benfica))
Sem dias com valor zero.

Contador: count_11 (#21 - Avenida dos Combatentes)
Sem dias com valor zero.

Contador: count_12 (#23 - Ciclovia Ribeirinha (P25 Abril /Café In))
Sem dias com valor zero.

Contador: count_13 (#07 - Jardim do

In [ ]:
count_cols = [col for col in df_daily.columns if col.startswith('count_')]

# Comparar os valores preenchidos com os originais: 1 se mantido, 0 se substituído
df_indicador = (df_daily[count_cols] == df_daily_filled[count_cols]).astype(int)

# Renomear colunas para indicar que são indicadores de substituição
df_indicador = df_indicador.add_prefix("new_")



In [57]:
print(df_indicador.head())
df_indicador.loc['2022-08-18']


            new_count_2  new_count_3  new_count_4  new_count_5  new_count_6  \
detected                                                                      
2021-08-01            1            1            1            1            1   
2021-08-02            1            1            1            1            1   
2021-08-03            1            1            1            1            1   
2021-08-04            1            1            1            1            1   
2021-08-05            1            1            1            1            1   

            new_count_7  new_count_8  new_count_9  new_count_10  new_count_11  \
detected                                                                        
2021-08-01            1            1            1             1             1   
2021-08-02            1            1            1             1             1   
2021-08-03            1            1            1             1             1   
2021-08-04            1            1     

new_count_2     1
new_count_3     0
new_count_4     0
new_count_5     1
new_count_6     0
new_count_7     0
new_count_8     0
new_count_9     0
new_count_10    0
new_count_11    1
new_count_12    0
new_count_13    0
new_count_14    1
new_count_15    0
new_count_16    1
new_count_17    0
new_count_19    0
new_count_20    0
new_count_21    0
new_count_22    0
new_count_24    0
new_count_25    0
new_count_26    1
new_count_28    0
new_count_29    0
new_count_30    0
new_count_31    0
new_count_32    0
new_count_33    0
new_count_34    1
new_count_35    1
Name: 2022-08-18 00:00:00, dtype: int64